In [1]:
# ---------- AUTO-DETECCIÓN DE HEADER + MODELO SUPERVISADO ----------
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

excel_file = "MAESTRO_DE_NOTAS_0105975569_20250620022145.xlsx"
nota_threshold = 70    # ajusta según tu escala (70/100) o pon 11 si tu escala es 0-20
test_size = 0.2
random_state = 42
use_class_weight_balanced = True

# 1) Leer sin header para inspeccionar filas
tmp = pd.read_excel(excel_file, header=None)

# Mostrar algunas filas para debug (comenta si molesta)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)
print("Primeras 15 filas (sin header):")
display(tmp.head(15))

# 2) Intentar detectar automáticamente la fila que contiene encabezados reales
# Buscaremos filas que contengan varias palabras clave presentes en tu archivo
keywords = ['nota', 'asistencia', 'estudiante', 'periodo', 'identificacion', 'estado', 'carrera']

header_row = None
for idx in range(min(30, len(tmp))):   # buscar solo en las primeras 30 filas
    row_vals = tmp.iloc[idx].astype(str).str.lower().tolist()
    hits = sum(1 for k in keywords if any(k in str(v) for v in row_vals))
    # Si la fila contiene al menos 2 coincidencias de keywords, la tomamos como header
    if hits >= 2:
        header_row = idx
        break

if header_row is None:
    print("\nNo se detectó automáticamente la fila de encabezado. Intenta buscar manualmente en las primeras 30 filas.")
    # Para ayudar, imprimimos las primeras 30 filas en una forma compacta para que indiques el índice
    display(tmp.head(30))
    raise SystemExit("No se detectó header automático. Indica cuál fila contiene los encabezados (índice).")

print(f"\nFila detectada como header: {header_row} (leeré el Excel con header={header_row})")

# 3) Leer nuevamente usando header_row como header
df = pd.read_excel(excel_file, header=header_row)

# 4) Normalizar nombres y limpiar columnas vacías / Unnamed
df.columns = df.columns.astype(str).str.strip()
df = df.dropna(axis=1, how='all')
df = df.loc[:, ~df.columns.str.contains("^Unnamed", na=False)]

print("\nColumnas detectadas después de ajustar header:")
print(df.columns.tolist())
print("\nNúmero de filas:", len(df))

# 5) Crear etiqueta Aprobado (preferir 'Estado' si existe)
if 'Estado' in df.columns and df['Estado'].astype(str).str.upper().isin(['APROBADO']).any():
    df['Aprobado'] = df['Estado'].apply(lambda x: 1 if str(x).strip().upper() == 'APROBADO' else 0)
    print("\nEtiqueta creada desde 'Estado'.")
else:
    # normalizar nombre si hay 'Nota Final' con mayúscula
    if 'Nota Final' in df.columns and 'Nota final' not in df.columns:
        df.rename(columns={'Nota Final':'Nota final'}, inplace=True)
    if 'Nota final' not in df.columns:
        print("\nNo encontré columna 'Nota final' ni 'Estado' tras relectura.")
        print("Columnas actuales:", df.columns.tolist())
        raise KeyError("No se encuentra 'Nota final' ni 'Estado'. Revisa el nombre exacto de la columna.")
    df['Nota final'] = pd.to_numeric(df['Nota final'], errors='coerce')
    df = df.dropna(subset=['Nota final'])
    df['Aprobado'] = (df['Nota final'] >= nota_threshold).astype(int)
    print(f"\nEtiqueta creada por umbral: Nota final >= {nota_threshold} -> Aprobado=1")

# 6) Seleccionar features
num_cols = [c for c in ['Asistencia', 'Nota final'] if c in df.columns]
cat_candidates = ['Nivel', 'Carrera', 'Asignatura', 'Paralelo', 'Tipo Ingreso', 'Estado Matrícula']
cat_cols = [c for c in cat_candidates if c in df.columns]
# añadir otras categóricas con pocos valores
auto_cat = [c for c in df.select_dtypes(include=['object']).columns if c not in cat_cols + ['Estudiante','Nombre docente','Cédula docente','Identificacion']]
auto_cat = [c for c in auto_cat if df[c].nunique() < 50]
for c in auto_cat:
    if c not in cat_cols:
        cat_cols.append(c)

print("\nNuméricas usadas:", num_cols)
print("Categóricas usadas:", cat_cols)

# 7) Preprocesador y pipeline
numeric_transformer = Pipeline([('scaler', StandardScaler())]) if num_cols else None
categorical_transformer = Pipeline([('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))]) if cat_cols else None

transformers = []
if num_cols:
    transformers.append(('num', numeric_transformer, num_cols))
if cat_cols:
    transformers.append(('cat', categorical_transformer, cat_cols))

preprocessor = ColumnTransformer(transformers=transformers, remainder='drop')

clf = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000,
                                      class_weight='balanced' if use_class_weight_balanced else None,
                                      random_state=random_state))
])

# 8) Train/test split y entrenamiento
X = df[num_cols + cat_cols] if (num_cols + cat_cols) else pd.DataFrame(index=df.index)
y = df['Aprobado'].astype(int)

if X.shape[1] == 0:
    raise ValueError("No se detectaron features numéricas ni categóricas para entrenar. Revisa las columnas del archivo.")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=random_state)
clf.fit(X_train, y_train)

# 9) Métricas
y_pred = clf.predict(X_test)
y_proba = None
try:
    y_proba = clf.predict_proba(X_test)[:,1]
except:
    pass

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)
auc = roc_auc_score(y_test, y_proba) if (y_proba is not None and len(np.unique(y_test))==2) else None

print(f"\nAccuracy: {acc:.4f}")
if auc is not None:
    print(f"AUC: {auc:.4f}")
print("\nMatriz de confusión:\n", cm)
print("\nReporte de clasificación:\n", report)

# 10) Mostrar matriz
plt.figure(figsize=(5,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.xlabel('Predicho')
plt.ylabel('Real')
plt.show()

# 11) Coeficientes / interpretación
feature_names = []
if num_cols:
    feature_names.extend(num_cols)
if cat_cols:
    ohe = clf.named_steps['preprocessor'].named_transformers_['cat'].named_steps['onehot']
    try:
        ohe_names = list(ohe.get_feature_names_out(cat_cols))
    except:
        ohe_names = []
        for i, col in enumerate(cat_cols):
            cats = ohe.categories_[i]
            ohe_names += [f"{col}_{str(c)}" for c in cats]
    feature_names.extend(ohe_names)

coefs = clf.named_steps['classifier'].coef_[0]
coef_df = pd.DataFrame({'feature': feature_names, 'coef': coefs})
coef_df['abs_coef'] = coef_df['coef'].abs()
coef_df = coef_df.sort_values(by='abs_coef', ascending=False)
coef_df['odds_ratio'] = np.exp(coef_df['coef'])

print("\nTop 20 features por impacto (coef absoluto):")
display(coef_df.head(20))

# 12) Guardar modelo
joblib.dump(clf, "logreg_model_aprobado_autodetect.joblib")
print("\nModelo guardado como 'logreg_model_aprobado_autodetect.joblib'")

# -----------------------------------------------------------------




Primeras 15 filas (sin header):


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INSTITUTO SUPERIOR TECNOLÓGICO DEL AZUAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,REPORTE MAESTRO DE NOTAS,NaN,NaN,NaN,NaN,Fecha Reporte:,NaN,NaN,NaN,NaN,NaN,NaN,VIERNES 20 JUNIO 2025 02:21 PM,NaN,NaN,NaN,NaN
3,NaN,Periodo,Paralelo,Identificacion,Estudiante,Carrera,Nivel,Asignatura,Num_matricula,NaN,Asistencia,Nota final,Estado,NaN,Estado Matrícula,Tipo Ingreso,Cédula docente,Nombre docente
4,NaN,2020-2P,B,1104826365,ABRIGO ZAPATA KARINA CECILIA,CDI-CENTRO DE IDIOMAS,PRIMERO,ING01-INGLÉS A1,0,NaN,91.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0101685428,CLARA CECILIA CLAVIJO CLAVIJO
5,NaN,2020-2P,L,1104826365,ABRIGO ZAPATA KARINA CECILIA,TAF-TECNOLOGIA SUPERIOR EN ASESORIA FINANCIERA,PRIMERO,TAF-116-ATENCIÓN AL CLIENTE,0,NaN,100.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0105003198,VERONICA MARIBEL OCHOA CALDERON
6,NaN,2020-2P,L,1104826365,ABRIGO ZAPATA KARINA CECILIA,TAF-TECNOLOGIA SUPERIOR EN ASESORIA FINANCIERA,PRIMERO,TAF-117-COMUNICACIÓN ORAL Y ESCRITA,0,NaN,100.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0104549159,JAIME GEOVANY LOJA BUESTAN
7,NaN,2020-2P,L,1104826365,ABRIGO ZAPATA KARINA CECILIA,TAF-TECNOLOGIA SUPERIOR EN ASESORIA FINANCIERA,PRIMERO,TAF-114-MANEJO DE CUENTAS,0,NaN,100.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0103280152,NORMA ELIZABETH VELECELA ABAMBARI
8,NaN,2020-2P,L,1104826365,ABRIGO ZAPATA KARINA CECILIA,TAF-TECNOLOGIA SUPERIOR EN ASESORIA FINANCIERA,PRIMERO,TAF-113-MARCO ECONÓMICO,0,NaN,100.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0103970851,MONICA ELIZABETH COBOS ROJAS
9,NaN,2020-2P,L,1104826365,ABRIGO ZAPATA KARINA CECILIA,TAF-TECNOLOGIA SUPERIOR EN ASESORIA FINANCIERA,PRIMERO,TAF-112-MARCO JURÍDICO DEL SECTOR FINANCIERO E...,0,NaN,100.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0103924692,MARCO PATRICIO SAMANIEGO DUMAS



Fila detectada como header: 3 (leeré el Excel con header=3)

Columnas detectadas después de ajustar header:
['Periodo', 'Paralelo', 'Identificacion', 'Estudiante', 'Carrera', 'Nivel', 'Asignatura', 'Num_matricula', 'Asistencia', 'Nota final', 'Estado', 'Estado Matrícula', 'Tipo Ingreso', 'Cédula docente', 'Nombre docente']

Número de filas: 4167

Etiqueta creada desde 'Estado'.

Numéricas usadas: ['Asistencia', 'Nota final']
Categóricas usadas: ['Nivel', 'Carrera', 'Asignatura', 'Paralelo', 'Tipo Ingreso', 'Estado Matrícula', 'Periodo', 'Estado']


TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [10]:
# ==========================================
# 3. Crear variable objetivo (Aprobado/Reprobado)
#    Supongamos que Nota_final es la columna.
# ==========================================
df["Aprobado"] = df["Nota final"].apply(lambda x: 1 if x >= 7 else 0)



KeyError: 'Nota final'

In [15]:
print(df.columns.tolist())


['REPORTE MAESTRO DE NOTAS', 'Fecha Reporte:']


In [18]:
temp = pd.read_excel(excel_file, header=None)
temp.head(15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INSTITUTO SUPERIOR TECNOLÓGICO DEL AZUAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,REPORTE MAESTRO DE NOTAS,NaN,NaN,NaN,NaN,Fecha Reporte:,NaN,NaN,NaN,NaN,NaN,NaN,VIERNES 20 JUNIO 2025 02:21 PM,NaN,NaN,NaN,NaN
3,NaN,Periodo,Paralelo,Identificacion,Estudiante,Carrera,Nivel,Asignatura,Num_matricula,NaN,Asistencia,Nota final,Estado,NaN,Estado Matrícula,Tipo Ingreso,Cédula docente,Nombre docente
4,NaN,2020-2P,B,1104826365,ABRIGO ZAPATA KARINA CECILIA,CDI-CENTRO DE IDIOMAS,PRIMERO,ING01-INGLÉS A1,0,NaN,91.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0101685428,CLARA CECILIA CLAVIJO CLAVIJO
5,NaN,2020-2P,L,1104826365,ABRIGO ZAPATA KARINA CECILIA,TAF-TECNOLOGIA SUPERIOR EN ASESORIA FINANCIERA,PRIMERO,TAF-116-ATENCIÓN AL CLIENTE,0,NaN,100.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0105003198,VERONICA MARIBEL OCHOA CALDERON
6,NaN,2020-2P,L,1104826365,ABRIGO ZAPATA KARINA CECILIA,TAF-TECNOLOGIA SUPERIOR EN ASESORIA FINANCIERA,PRIMERO,TAF-117-COMUNICACIÓN ORAL Y ESCRITA,0,NaN,100.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0104549159,JAIME GEOVANY LOJA BUESTAN
7,NaN,2020-2P,L,1104826365,ABRIGO ZAPATA KARINA CECILIA,TAF-TECNOLOGIA SUPERIOR EN ASESORIA FINANCIERA,PRIMERO,TAF-114-MANEJO DE CUENTAS,0,NaN,100.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0103280152,NORMA ELIZABETH VELECELA ABAMBARI
8,NaN,2020-2P,L,1104826365,ABRIGO ZAPATA KARINA CECILIA,TAF-TECNOLOGIA SUPERIOR EN ASESORIA FINANCIERA,PRIMERO,TAF-113-MARCO ECONÓMICO,0,NaN,100.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0103970851,MONICA ELIZABETH COBOS ROJAS
9,NaN,2020-2P,L,1104826365,ABRIGO ZAPATA KARINA CECILIA,TAF-TECNOLOGIA SUPERIOR EN ASESORIA FINANCIERA,PRIMERO,TAF-112-MARCO JURÍDICO DEL SECTOR FINANCIERO E...,0,NaN,100.00,NaN,RETIRADO,NaN,APROBADO,NORMAL,0103924692,MARCO PATRICIO SAMANIEGO DUMAS
